# Import requerment library

In [1]:
import re
import string
import numpy as np
import pandas as pd

# Import fakeand real data

In [2]:
fake_data = pd.read_csv(r'I:\Deep Learning data\fake_news_dataset\Fake.csv')

In [3]:
fake_data.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true_data = pd.read_csv(r'I:\Deep Learning data\fake_news_dataset\True.csv')

In [5]:
true_data.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
fake_data['class'] = 0
true_data['class'] = 1

In [38]:
fake_data.shape, true_data.shape

((23481, 5), (21417, 5))

In [39]:
df_fake_manual_testing = fake_data.tail(10)
for i in range(23480, 23470,-1):
    fake_data.drop([i],axis=0,inplace=True)
df_true_manual_testing = true_data.tail(10)
for i in range(21416, 21406,-1):
    true_data.drop([i],axis=0,inplace=True)

In [41]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing],axis=0)
df_manual_testing.to_csv(r'I:\Deep Learning data\fake_news_dataset\manual_testing1.csv')

In [7]:
df_marge = pd.concat([fake_data,true_data],axis=0)

In [8]:
df_marge.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [9]:
df = df_marge.drop(['title','subject','date'],axis=1)
df.head()


,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


# Randomly shuffling the dataframe

In [10]:
df = df.sample(frac=1)
df.head()

,text,class
10354,Judicial Watch has released documents obtained...,0
2260,"LONDON, (Reuters) - The chairman of a congress...",1
22754,Our weekly documentary film curated by our edi...,0
8951,Anyone who watched Barack Obama break into tea...,0
8420,The nightmare that is Donald Trump just keeps ...,0


In [11]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [12]:
df.isnull().sum()

text     0
class    0
dtype: int64

# Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links

In [13]:
def word_dorp(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [14]:
df['text'] = df['text'].apply(word_dorp)

In [15]:
df.head()

,text,class
0,judicial watch has released documents obtained...,0
1,london reuters the chairman of a congress...,1
2,our weekly documentary film curated by our edi...,0
3,anyone who watched barack obama break into tea...,0
4,the nightmare that is donald trump just keeps ...,0


In [16]:
X = df["text"]
y = df["class"]

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

# Convert text to vectors

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
vectorizer = TfidfVectorizer()

In [20]:
X_train =vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Various type of model implementation

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [22]:
lr = LogisticRegression()

In [23]:
lr.fit(X_train,y_train)

LogisticRegression()

In [24]:
y_pred = lr.predict(X_test)

In [25]:
print(classification_report(y_pred,y_test))
print(accuracy_score(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5870
           1       0.99      0.98      0.99      5355

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225

0.9871714922048997
[[5809   61]
 [  83 5272]]


# Decision Tree Classification


In [26]:
from sklearn.tree import DecisionTreeClassifier

In [27]:
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)

DecisionTreeClassifier()

In [28]:
y_pred = dt.predict(X_test)

In [29]:
print(classification_report(y_pred,y_test))
print(accuracy_score(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5893
           1       1.00      1.00      1.00      5332

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225

0.9958129175946547
[[5869   24]
 [  23 5309]]


# Gradient Boosting Classifier

In [30]:
from sklearn.ensemble import GradientBoostingClassifier

In [31]:
gbc = GradientBoostingClassifier(random_state=0)
gbc.fit(X_train,y_train)

GradientBoostingClassifier(random_state=0)

In [32]:
y_pred = gbc.predict(X_test)

In [33]:
print(classification_report(y_pred,y_test))
print(accuracy_score(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5869
           1       1.00      0.99      0.99      5356

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225

0.99456570155902
[[5850   19]
 [  42 5314]]


# Model Testing With Manual Entry

In [34]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "True News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(word_dorp) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorizer.transform(new_x_test)
    pred_LR = lr.predict(new_xv_test)
    pred_DT = dt.predict(new_xv_test)
    pred_GBC = gbc.predict(new_xv_test)
  

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} ".format(output_lable(pred_LR[0]), 
                                                                                          output_lable(pred_DT[0]), 
                                                                                          output_lable(pred_GBC[0])))

In [ ]:
news = str(input())
manual_testing(news)